## Logistic Regression Multiclass

In [40]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
% matplotlib inline

In [66]:
class LogisticRegression:
        
    def thres(self,n):
        if(n>=0.5):
            return 1
        else:
            return 0
    
    def hyp(self):
        pp= -1 * np.dot(self.X, self.theta)
        prob= (1.0 / (1 + (np.e **(pp))))
        newf=  np.vectorize(self.thres)
        newp= newf(prob)
        return prob
    
        
    def get_cost(self):
        aa= np.dot(np.log(self.hyp().transpose()), self.y)
        bb= np.dot((np.log(1 - self.hyp().transpose())), (1- self.y))
        coss= aa + bb
        mcost = ((-1.0/self.m) * sum(coss))
        return mcost
    
    def fscale(self):
        self.X= self.X / (self.X.max(axis= 0) - self.X.min(axis= 0))
        self.y= self.y / (max(self.y) - min(self.y))
        
    def computeStandard(self):
        from sklearn.linear_model import LogisticRegression as LR
        self.logreg= LR()
        self.logreg.fit(self.X, self.y)

    def process_col(self, col):
        col= col.values
        return col[None].transpose()
        
    def preprocess(self, X, y, alpha):
        (self.m, self.n)= X.shape
        self.columns= X.columns
        self.X= X.values
        self.ny= pd.get_dummies(y)
        self.nc= len(y.unique())
        self.nccols= y.unique()
        self.thetas= [np.zeros(self.n+1)[None].transpose()]*self.nc
        
        #self.fscale()
        #self.computeStandard()
        self.n+=1
        self.X= np.insert(self.X, 0, 1, axis= 1)
        self.alpha= alpha
       
          
    def fit(self, X, y, alpha= 0.1):
        self.preprocess(X, y, alpha)
        
        for i in range(self.nc):
            self.y= self.ny[self.nccols[i]]
            self.y= self.process_col(self.y)
            self.theta= self.thetas[i]
            pc= math.inf
            change= 1
            iteration=0
            #print("Change is ",change)
            while(change>0.0000001 and iteration<=100):
                der = (self.hyp() - self.y).transpose()
                der = np.dot(der , self.X) * self.alpha / self.m
                self.theta= self.theta - der.transpose()
                self.cost= self.get_cost()
                change= pc - self.cost
                pc= self.cost
                iteration += 1
                if(iteration % 1000 == 0):
                    print("{}. Cost: {}, theta: {}".format(iteration, self.cost, self.theta.transpose()))
            #print("Change is ",change)
            print('+'*70, "\n{}. Final Cost: {}, theta: {}".format(iteration, self.cost, self.theta.transpose()))
        #print("sklearn: theta: ",self.logreg.intercept_, self.logreg.coef_,"\n")
        
            
    def predict(self, X):
        X= X.values
        X= np.insert(X, 0, 1, axis= 1)
        
        pp= -1 * np.dot(X, self.theta)
        prob= (1.0 / (1 + (np.e **(pp))))
        newf=  np.vectorize(self.thres)
        newp= newf(prob)
        return newp.transpose()[0], prob.transpose()[0]
    
    def accuracy(self, y_true, y_pred):
        return sum(y_true == y_pred) / len(y_true)

### Diabetes Dataset

In [67]:
data= pd.read_csv('Files/Iris.csv')
#mapper= {'Iris-versicolor':1, 'Iris-setosa':2, 'Iris-virginica':3}
#data['Species']= data['Species'].map(mapper)
#dumm= pd.get_dummies(data['Species'])
#data= pd.concat([data, dumm], axis=1)
feature_cols= ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
target_col= 'Species'
X= data[feature_cols]
y= data[target_col]
data.sample(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
149,150,5.9,3.0,5.1,1.8,Iris-virginica
72,73,6.3,2.5,4.9,1.5,Iris-versicolor
69,70,5.6,2.5,3.9,1.1,Iris-versicolor
9,10,4.9,3.1,1.5,0.1,Iris-setosa
56,57,6.3,3.3,4.7,1.6,Iris-versicolor


In [68]:
logreg= LogisticRegression()
logreg.fit(X, y, alpha=0.0000001)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
101. Final Cost: [0.69310727], theta: [[-1.68314999e-06 -1.26541925e-05 -3.91487949e-06 -1.40516964e-05
  -5.23154629e-06]]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
101. Final Cost: [0.69313186], theta: [[-1.68321421e-06 -9.52358778e-06 -6.09667067e-06 -4.63878582e-06
  -1.58891005e-06]]
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
2. Final Cost: [0.69314701], theta: [[-3.33333182e-08 -1.45133244e-07 -1.07133287e-07 -5.73327455e-09
   1.52000189e-08]]


In [9]:
logreg= LogisticRegression()
logreg.fit(X, y, alpha= 0.00025)

NameError: name 'LogisticRegression' is not defined

### Model Accuracy

In [7]:
y_pred, y_pred_prob= logreg.predict(X)
result= pd.DataFrame({"True":y, "Prediction":y_pred, "Probablity":y_pred_prob})
score= logreg.accuracy(y, y_pred)
print("Accuracy:",score)

Accuracy: 0.7486979166666666


### Null Accuracy

In [8]:
y.value_counts()[0] / len(y)

0.6510416666666666

### Standard Accuracy

In [10]:
from sklearn.linear_model import LogisticRegression as LR
lr= LR()
lr.fit(X, y)
y_pred= lr.predict(X)
from sklearn.metrics import accuracy_score
score= accuracy_score(y, y_pred)
print('Score',score)
print('theta:', lr.coef_)
print('intercept:', lr.intercept_)

Score 0.7747395833333334
theta: [[ 0.11322185  0.02873252 -0.01710249  0.0022031  -0.00049272  0.06198601
   0.00858668]]
intercept: [-5.74521571]


### To be continued...